# `Isotope` and `IsotopeQuantity` examples

First, some imports:

In [1]:
from becquerel.tools import Isotope, IsotopeQuantity, NeutronIrradiation
import datetime

## Isotope string formatting and properties with `Isotope`

Isotopes can be instantiated in any of a number of ways:

In [2]:
i1 = Isotope('Cs-137')
i2 = Isotope('137CS')
i3 = Isotope('Cs', 137)
i4 = Isotope('Cesium-137')
i5 = Isotope('137CAESIUM')
print(i1, i2, i3, i4, i5)
print(i1 == i2 == i3 == i4 == i5)

Cs-137 Cs-137 Cs-137 Cs-137 Cs-137
True


Isotopes have a string format method:

In [3]:
cs137 = Isotope('Cs-137')
print(cs137)
print('{:%n(%s)-%a%m}'.format(cs137))

Cs-137
Cesium(Cs)-137


Metastable isomers can be described:

In [4]:
tc99m = Isotope('Tc-99m')
print(tc99m)
print('{:%n-%a%m}'.format(tc99m))
hf178m2 = Isotope('Hf', 178, 'm2')
print(hf178m2)

Tc-99m
Technetium-99m
Hf-178m2


Finally, isotope properties such as half-life and natural abundance are available:

In [5]:
for a in range(37, 44):
    iso = Isotope('Potassium', a)
    print('')
    print('Isotope: {}'.format(iso))
    print('Spin-parity: {}'.format(iso.j_pi))
    if iso.abundance is not None:
        print('Abundance:   {:.2f}%'.format(iso.abundance))
    print('Stable?      {}'.format(iso.is_stable))
    if not iso.is_stable:
        print('Half-life:   {:.3e} years'.format(iso.half_life / 365.25 / 24 / 3600))
        print('Decay modes: {}'.format(iso.decay_modes))



Isotope: K-37
Spin-parity: 3/2+
Stable?      False
Half-life:   3.885e-08 years
Decay modes: (['EC'], [100.0])

Isotope: K-38
Spin-parity: 3+
Stable?      False
Half-life:   1.452e-05 years
Decay modes: (['EC'], [100.0])

Isotope: K-39
Spin-parity: 3/2+
Abundance:   93.26+/-0.00%
Stable?      True

Isotope: K-40
Spin-parity: 4-
Abundance:   0.01+/-0.00%
Stable?      False
Half-life:   1.248e+09 years
Decay modes: (['EC', 'B-'], [10.72, 89.28])

Isotope: K-41
Spin-parity: 3/2+
Abundance:   6.73+/-0.00%
Stable?      True

Isotope: K-42
Spin-parity: 2-
Stable?      False
Half-life:   1.409e-03 years
Decay modes: (['B-'], [100.0])

Isotope: K-43
Spin-parity: 3/2+
Stable?      False
Half-life:   2.544e-03 years
Decay modes: (['B-'], [100.0])


## Decay calculations with `IsotopeQuantity`

Let's see how we can use the IsotopeQuantity class to work with lab check sources, starting with this Cs-137 source from the lab. How strong is it now?

In [6]:
cs137_chk = IsotopeQuantity(cs137, date='2008-01-15', uci=11.13)
cs137_chk.uci_at()

8.00631769726367

In fact, we can input an inventory of sources and easily see what's still active, without looking up half lives. The isotopes will automatically be created with the `Isotope` class if they are given as strings.

In [7]:
ba133_chk = IsotopeQuantity('ba133', date='2008-01-15', uci=10.24)
cd109_chk = IsotopeQuantity('cd109', date='2008-01-15', uci=10.3)
co57_chk = IsotopeQuantity('co57', date='2008-01-15', uci=11.21)
co60_chk = IsotopeQuantity('co60', date='2008-01-15', uci=9.38)
mn54_chk = IsotopeQuantity('mn54', date='2008-01-15', uci=9.03)
na22_chk = IsotopeQuantity('na22', date='2008-01-15', uci=10.21)

for chk in (ba133_chk, cd109_chk, co57_chk, co60_chk, mn54_chk, na22_chk):
    print('{}: {:.3f} uCi'.format(chk.isotope, chk.uci_at()))

Ba-133: 4.004 uCi
Cd-109: 0.004 uCi
Co-57: 0.000 uCi
Co-60: 1.431 uCi
Mn-54: 0.000 uCi
Na-22: 0.226 uCi


There are a variety of ways to specify a source activity: Bq, number of atoms, or grams.

In [8]:
print(IsotopeQuantity('co60', date='2017-03-16 14:00:00', bq=3.7e4))
print(IsotopeQuantity('k40', date=datetime.datetime.now(), atoms=4.2e24))
print(IsotopeQuantity('u238', g=0.33))

37000.0 Bq of Co-60 (at 2017-03-16 14:00:00)
73926311.7915635 Bq of K-40 (at 2022-05-02 14:53:32.265827)
4104.821259246285 Bq of U-238 (at 2022-05-02 14:53:32.272388)


If the `date` argument is not specified, the reference date defaults to `datetime.datetime.now()`.

Of course, we can check activities at other times too, and check other quantities.

In [9]:
co60_chk.uci_at('2014-11-05')

3.8317515981908064

In [10]:
cd109_chk.bq_at()

149.24053998847538

In [11]:
cs137_chk.g_at(cs137_chk.ref_date)

1.2830528795547293e-07

In [12]:
co57_chk.atoms_at()

23101885.23242553

We can also evaluate the number of decays in a specific time interval, or the average activity in the interval.

(In the future, this will be useful for interfacing directly with a `Spectrum` object, and calibrating efficiency, for example.)

In [13]:
cs137_chk.decays_from('2017-05-22 12:00:00', '2017-05-22 12:05:00')

99595559.0625

In [14]:
cs137_chk.uci_from('2017-05-22 12:00:00', '2017-05-22 12:05:00')

8.972572888513515

Finally, we can also calculate times when the source will have decayed to a given level:

In [15]:
print(cs137_chk.time_when(uci=1))

2112-08-12 22:32:16.007576


## Calculating activations with `NeutronIrradiation`

Currently, capture cross sections are not scraped from the web, so for now they must be input by hand. 

Still, `NeutronIrradiation` can make our life easier.

In [16]:
irradiation_start = '2017-04-30 10:32:00'
irradiation_stop = '2017-04-30 11:32:00'
flux = 3.1e11
ni = NeutronIrradiation(irradiation_start, irradiation_stop, n_cm2_s=flux)
print(ni)

310000000000.0 n/cm2/s from 2017-04-30 10:32:00 to 2017-04-30 11:32:00


Note that timestamps can be specified as strings (which are parsed with `dateutil`) or `datetime.datetime` objects.

We can calculate how much of a sample will be activated:

In [17]:
iso = Isotope('Na-23')
sample = IsotopeQuantity(iso, g=0.014)
iso2 = Isotope('Na-24')
barns = 2.3   # making this up for now

na24_qty = ni.activate(barns, initial=sample, activated=iso2)
print(na24_qty)

11804946.779552134 Bq of Na-24 (at 2017-04-30 11:32:00)


We can also use a known irradiation product quantity to back-calculate how much was in the original sample:

In [18]:
activated_qty = IsotopeQuantity(iso2, date='2017-05-01', bq=103.2)
na23_qty = ni.activate(barns, initial=iso, activated=activated_qty)
print(na23_qty)

2.177625139319166e-07 g of Na-23


In [19]:
na23_qty.g_at()

2.177625139319166e-07